# A solvis demo with the NSHM Composite Solution (NEW filter API)

In [1]:
import os
import pathlib
import json
from IPython.display import GeoJSON

import nzshm_model as nm
import solvis

warning openquake module dependency not available, maybe you want to install
                with nzshm-model[openquake]


Running without `toshi` options


In [2]:
# prepare our NSHM composite solution
current_model = nm.get_model_version("NSHM_v1.0.4")
slt = current_model.source_logic_tree

In [3]:
# load the composite_solution and shoose the Cruslal FaultSystemSolution
csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
fss = csol._solutions['CRU']

In [4]:
# try our new filtering
from solvis.filter import FilterSubsectionIds, FilterRuptureIds

In [6]:
# helper = FaultSystemSolutionHelper(fss)
filt_sub_ids = FilterSubsectionIds(fss)
filt_rupt_ids = FilterRuptureIds(fss)

In [7]:
filt_sub_ids.for_parent_fault_names(['Aka Aka'])

{3, 4}

In [11]:
boo_boo_rids = filt_rupt_ids.for_parent_fault_names(['BooBoo', 'Masterton'])
mag_rids = filt_rupt_ids.for_magnitude(4.5, 7.5)
rate_rids = filt_rupt_ids.for_rupture_rate(1e-90)

print(len(mag_rids & boo_boo_rids))

subsections = filt_sub_ids.for_rupture_ids(set.intersection(set(mag_rids), set(rate_rids), set(boo_boo_rids)))
surfaces = fss.fault_surfaces()
filtered = surfaces[surfaces.FaultID.isin(subsections)]

filtered.shape

18


(56, 14)

In [13]:
frids = FilterRuptureIds(fss)
fsids = FilterSubsectionIds(fss)
my_rids = frids.for_parent_fault_names(['BooBoo', 'Masterton']) - \
    frids.for_magnitude(4.5, 7.5) & \
    frids.for_rupture_rate(1e-90)
    #, drop_zero_rates=False)

surfaces = fss.fault_surfaces()
subsections = FilterSubsectionIds(fss).for_rupture_ids(my_rids)
filtered = surfaces[surfaces.FaultID.isin(subsections)]

filtered.tail()

,FaultID,FaultName,DipDeg,Rake,LowDepth,UpDepth,DipDir,AseismicSlipFactor,CouplingCoeff,ParentID,ParentName,geometry,Target Slip Rate,Target Slip Rate StdDev
2287,2287,"Wharekauhau, Subsection 8",45.0,90.0,6.64,0.0,309.6,0.0,1.0,547,Wharekauhau,"POLYGON ((175.05471 -41.3651, 175.08969 -41.33...",0.000112,0.000363
2288,2288,"Wharekauhau, Subsection 9",45.0,90.0,6.64,0.0,309.6,0.0,1.0,547,Wharekauhau,"POLYGON ((175.08969 -41.33258, 175.12463 -41.3...",0.000112,0.000363
2289,2289,"Wharekauhau, Subsection 10",45.0,90.0,6.64,0.0,309.6,0.0,1.0,547,Wharekauhau,"POLYGON ((175.12463 -41.30005, 175.1308 -41.29...",0.000121,0.000299
2309,2309,"White Creek - buried, Subsection 0",70.0,70.0,19.92,0.0,119.6,0.0,1.0,551,White Creek - buried,"POLYGON ((171.7406 -42.4659, 171.7581 -42.4488...",0.000315,0.001005
2310,2310,"White Creek - buried, Subsection 1",70.0,70.0,19.92,0.0,119.6,0.0,1.0,551,White Creek - buried,"POLYGON ((171.80531 -42.39574, 171.806 -42.393...",0.000135,0.000654


## display the surface geometry of a fault

In [14]:
surfaces = fss.fault_surfaces()
mro_surface = surfaces[surfaces.ParentName=="Masterton"]

# A fault called `Masterton` having 4 subsections
mro_surface # display the dataframe

,FaultID,FaultName,DipDeg,Rake,LowDepth,UpDepth,DipDir,AseismicSlipFactor,CouplingCoeff,ParentID,ParentName,geometry,Target Slip Rate,Target Slip Rate StdDev
1125,1125,"Masterton, Subsection 0",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.7331 -40.9283, 175.7213 -40.9359...",0.000652,0.000577
1126,1126,"Masterton, Subsection 1",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.66992 -40.93846, 175.60882 -40.9...",0.000788,0.000592
1127,1127,"Masterton, Subsection 2",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.60882 -40.96057, 175.5877 -40.96...",0.000652,0.000530
1128,1128,"Masterton, Subsection 3",65.0,-160.0,12.88,0.0,164.0,0.0,1.0,257,Masterton,"POLYGON ((175.54348 -40.97084, 175.5408 -40.97...",0.000518,0.000468


In [16]:
# plot the masterton surfaces
GeoJSON(json.loads(mro_surface.to_json()))

<IPython.display.GeoJSON object>

In [ ]:
### get ruptures passing through a fault

In [19]:
# mro_ruptures = helper.ruptures_for_parent_fault_names(['Masterton'])
mro_ruptures = filt_rupt_ids.for_parent_fault_names(['Masterton'])

### get rupture fault sections (rs) with rates for those ruptures

In [20]:
df0 = fss.rs_with_rupture_rates
mro_df = df0[df0["Rupture Index"].isin(mro_ruptures)]
mro_df

key_0 fault_system  Rupture Index  rate_max  \
fault_system Rupture Index                                                 
CRU          209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
             209026         209026          CRU         209026  0.000023   
...                            ...          ...            ...       ...   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   
             391758         391758          CRU         391758  0.000100   

                                rate_min  rate_count  rate_weighted_mean  \
fault_system Rupture Index                                                 
CRU          209026         3.083809e-07          12            0.000003   
             209026         3.083809e-07          12            0.000003   
             209026         3.083809e-07          12            0.000003   
             209026         3.083809e-07          12            0.000003   
             209026         3.083809e-07          12            0.000003   
...                                  ...         ...                 ...   
             391758         2.606850e-05          18            0.000032   
             391758         2.606850e-05          18            0.000032   
             391758         2.606850e-05          18            0.000032   
             391758         2.606850e-05          18            0.000032   
             391758         2.606850e-05          18            0.000032   

                            Magnitude  Average Rake (degrees)    Area (m^2)  \
fault_system Rupture Index                                                    
CRU          209026          7.998682               156.75558  6.290518e+09   
             209026          7.998682               156.75558  6.290518e+09   
             209026          7.998682               156.75558  6.290518e+09   
             209026          7.998682               156.75558  6.290518e+09   
             209026          7.998682               156.75558  6.290518e+09   
...                               ...                     ...           ...   
             391758          7.152529               167.70062  8.964618e+08   
             391758          7.152529               167.70062  8.964618e+08   
             391758          7.152529               167.70062  8.964618e+08   
             391758          7.152529               167.70062  8.964618e+08   
             391758          7.152529               167.70062  8.964618e+08   

                            Length (m)  section  
fault_system Rupture Index                       
CRU          209026         292585.340    725.0  
             209026         292585.340    724.0  
             209026         292585.340    723.0  
             209026         292585.340    722.0  
             209026         292585.340    721.0  
...                                ...      ...  
             391758          53762.266   2106.0  
             391758          53762.266   2107.0  
             391758          53762.266   1127.0  
             391758          53762.266   1207.0  
             391758          53762.266   1206.0  

[275 rows x 12 columns]

In [23]:
print(f'the `Masterton` fault has {len(mro_df["Rupture Index"].unique())} unique ruptures...')
print()
mro_ids = list(mro_df["Rupture Index"].unique())

the `Masterton` fault has 19 unique ruptures...



In [22]:
# let's pick one rupture and display it
surface = fss.rupture_surface(mro_ids[5])
GeoJSON(json.loads(surface.to_json()))

<IPython.display.GeoJSON object>